## Sparsity-Agnostic Linear Bandits with Adaptive Adversaries

This code is the official implementation of 'Sparsity-Agnostic Linear Bandits with Adaptive Adversaries.'

To proceed, simply press 'shift+enter' for each cell. 

### Requirements
The following cell includes all the required packages for this code.

In [1]:
from functools import wraps
from typing import List, Tuple
import numpy as np
import scipy.sparse as sp
from scipy.special import softmax
from numpy import linalg as LA
import argparse
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import time
import sys
sys.argv = ['']

### Algorithm class
**Bandit_Algo** is a parent class for all bandit algorithms.

All algorithms will have the following methods:

- *\_\_init\_\_*: is a constructor to initialize(assign values) to the data members of the class when an algorithm class is created.
- *sample*: is a method that will be called when the algorithm 'samples' an actoin based on the history. 
- *update*: will be called right after *sample* method to update its information.

All algorithms will take the following parameters as a default:
- $\theta$: hidden reward parameter. This is mainly used for the \_Known suffix experiment, where we assume that the algorithm knows the sparsity level of $\theta_*$ in advance. 
- $d$: Dimensionality of this linear bandit problem.
- $T$: Total number of iterations
- $\delta$: Error probability. 
- $\lambda$: regularization parameter for the $\hat{\theta}_t$. Throughout our paper and this experiment, this will be always 1. 
- $\sigma$: The width of the uniform distribution of the noise. Noise follows $Unif(-\sigma, \sigma)$ distribution.


In [2]:
class Bandit_Algo():
    def __init__(self,theta,d,T,delta,lamb, sigma):
        self.theta=theta
        self.T=T
        self.cum_regret=np.zeros(T)
        self.delta=delta
        self.Vt=lamb*np.identity(d)
        self.hat_theta=np.zeros(d)
    def sample(self, A):
        return 0;
    def update(self):
        return 0;

**SparseLinUCB** is Algorithm 1 in our main paper.

Basically, we implemented the pseudocode from the paper, and the parameters added from the parent class are as follows:

 - *coin_dist*: is a parameter which represents the model selection distribution
   - When coin_dist takes a list, the input list becomes the model selection distribution.
   - coin_dist==-1 corresponds to *\_Theory* suffix, meaning $q_s = \Theta(2^{-s})$ for $s=0,\ldots,n$.
   - coin_dist==-2 corresponds to *\_Unif* suffix, meaning $q_s = \frac{1}{n}$ for $s=0, \ldots, n$. 
   - coin_dist==-3 corresponds to *\_Known* suffix, meaning $q_s = \mathbb{1}\{s=o\}$ when the optimal index $o$ (for the given $S$) is known in advance.

In [3]:
class SparseLinUCB(Bandit_Algo):
    def __init__(self,theta,d,T,delta=0.01, lamb=1, coin_dist=-1, c=1):        
        self.theta=theta
        self.s=np.sum(theta!=0)
        self.o=int(np.ceil(np.log2(self.s))+1)
        self.T=T
        self.cum_regret=np.zeros(T)
        self.delta=delta
        self.Vt=np.identity(d)*lamb
        self.VtInv=LA.inv(self.Vt)
        self.hat_theta=np.zeros(d)
        self.t=0
        self.d=d
        self.n=int(np.ceil(np.log2(d)*c))+2 # For example, d=16=2^4, we need greedy, s=1=2^0, s=2=2^1, s=4=2^2, s=8=2^3, s=16=2^4
        self.alpha=np.zeros(self.n)
        self.coin_dist=np.ones(self.n)
        self.Xr=np.zeros(d)
        for i in range(self.n):
            self.alpha[i]=2**(i-1)
            if coin_dist==-1:
                self.coin_dist[i]=1/self.alpha[i]
            elif coin_dist==-2:
                self.coin_dist[i]=1/self.n
            elif coin_dist==-3:
                self.coin_dist[i]=(i==self.o)
            else:
                self.coin_dist=np.array(coin_dist)
        self.coin_dist=self.coin_dist/np.sum(self.coin_dist)
        self.alpha[0]=0

    def sample(self,A):
        alpha_i=self.alpha[np.random.choice(self.n, p=self.coin_dist)]
        N_a, _ = np.shape(A)
        UCB=np.zeros(N_a)
        norms=np.sqrt(np.diag(A@self.VtInv@A.T))
        for i in range(N_a):
            UCB[i]=np.dot(self.hat_theta,A[i])+np.sqrt(alpha_i*np.log(self.t+1))*norms[i]
        return A[np.argmax(UCB)]

    def update(self, A, t, a_t, r_t):
        self.Vt=self.Vt+np.outer(a_t, a_t)
        self.VtInv=self.VtInv - (np.outer(self.VtInv@a_t, self.VtInv@a_t))/(1+a_t@self.VtInv@a_t) #Sherman–Morrison formula
        self.Xr=self.Xr+a_t *r_t
        self.hat_theta = self.VtInv@self.Xr
        best_reward=np.max(A@self.theta)
        instant_reg=best_reward-np.dot(self.theta, a_t)
        self.t=t
        if t==0:
            self.cum_regret[t]=instant_reg
        else:
            self.cum_regret[t]=self.cum_regret[t-1]+instant_reg


**AdaLinUCB** is Algorithm 2 in our main paper. 

There are two noteworthy parameters added from the parent class:

 - *coin_dist*: is a parameter which represents the first model selection distribution, corresponding to $Z_t$ in our pseudocode.
 - *prior*: is an initial model selection distribution of Exp3, corresponding to $q$ (see second paragraph of the Section 5 for its explanation). 
    - 'U': corresponds to *\_Unif* suffix, meaning $q_s = \Theta(2^{-s})$ for $s=0,\ldots,n$.
    - 'T': corresponds to *\_Theory* suffix, meaning $q_s = \frac{1}{n}$ for $s=0, \ldots, n$. 

In [4]:
class AdaLinUCB(Bandit_Algo):
    def __init__(self,theta,d,T,delta=0.01, lamb=1, coin_dist=0, c=1, prior='U'):        
        self.theta=theta
        self.T=T
        self.cum_regret=np.zeros(T)
        self.delta=delta
        self.Vt=np.identity(d)*lamb
        self.VtInv=LA.inv(self.Vt)
        self.hat_theta=np.zeros(d)
        self.t=0
        self.d=d
        self.n=int(np.ceil(np.log2(d)*c))+2
        self.eta=2*np.sqrt(np.log(self.n)/(1*self.n))
        self.alpha=np.zeros(self.n)
        self.coin_dist=np.ones(self.n)
        self.Xr=np.zeros(d)
        
        self.prior=np.ones(self.n)
        for i in range(self.n):
            if prior=='T':
                self.prior[i]=2**(-i)
            self.alpha[i]=2**(i-1) ####Original version
            self.alpha[0]=0
        self.coin_dist=[1-coin_dist, coin_dist]
        self.S=np.zeros(self.n)
        self.prior=self.prior/np.sum(self.prior)
        self.exp_dist=self.prior*softmax(self.eta*self.S)
        self.exp_dist=self.exp_dist/np.sum(self.exp_dist)
        self.first_coin=0
        self.second_coin=0

    def sample(self,A):
        self.first_coin=np.random.choice(2, p=self.coin_dist)
        N_a, _ = np.shape(A)
        UCB=np.zeros(N_a)
        norms=np.sqrt(np.diag(A@self.VtInv@A.T))

        if self.first_coin==1:
            for i in range(N_a):
                UCB[i]=np.dot(self.hat_theta,A[i])+np.sqrt(self.alpha[-1]*np.log(self.t+1))*norms[i]
        else:
            self.exp_dist=self.prior*softmax(self.eta*self.S)
            self.exp_dist=self.exp_dist/np.sum(self.exp_dist)
            self.second_coin=np.random.choice(self.n, p=self.exp_dist)
            self.alpha_i=self.alpha[self.second_coin]
            for i in range(N_a):
                UCB[i]=np.dot(self.hat_theta,A[i])+np.sqrt(self.alpha_i*np.log(self.t+1))*norms[i]

        return A[np.argmax(UCB)]

    def update(self, A, t, a_t, r_t):
        if self.first_coin==0:     
            self.S[self.second_coin]=self.S[self.second_coin]-(2-r_t)/(4*(self.exp_dist[self.second_coin])) #Omitted +1 summation for everyone           
            self.exp_dist=self.prior*softmax(self.eta*self.S)
            self.exp_dist=self.exp_dist/np.sum(self.exp_dist)
        self.Vt=self.Vt+np.outer(a_t, a_t)
        self.t=t
        self.VtInv=self.VtInv - (np.outer(self.VtInv@a_t, self.VtInv@a_t))/(1+a_t@self.VtInv@a_t) # Sherman–Morrison formula
        self.Xr=self.Xr+a_t *r_t
        self.hat_theta = self.VtInv@self.Xr
        best_reward=np.max(A@self.theta)
        self.eta=2*np.sqrt(np.log(self.n)/((t+1)*self.n))
        instant_reg=best_reward-np.dot(self.theta, a_t)
        if t==0:
            self.cum_regret[t]=instant_reg
        else:
            self.cum_regret[t]=self.cum_regret[t-1]+instant_reg


**OFUL** is a class that implements the algorithm devised in the paper ['Improved Algorithms for Linear Stochastic Bandits'](https://papers.nips.cc/paper_files/paper/2011/file/e1d5be1c7f2f456670de3d53c7b54f4a-Paper.pdf)

- This code is based on the log-determinant version of the confidence set. See Appendix E.3 for details. 

In [5]:
class OFUL(Bandit_Algo):
    def __init__(self,theta,d,T,delta=0.01, sigma=1, lamb=1, radi=False):        
        self.theta=theta
        self.T=T
        self.t=1
        self.radi=radi
        self.sigma=sigma
        self.cum_regret=np.zeros(T)
        self.delta=delta
        self.Vt=np.identity(d)*lamb
        self.lamb=lamb
        self.VtInv=LA.inv(self.Vt)
        self.hat_theta=np.zeros(d)
        self.detV=1
        self.d=d
        if radi:
            self.beta=self.sigma*np.sqrt(self.d*np.log(1+(self.t)/(self.d*self.lamb)))+np.sqrt(self.lamb)*LA.norm(self.theta)
        else:
            self.beta=self.sigma*np.sqrt(np.log(self.detV/(self.delta**2)))+np.sqrt(self.lamb)*LA.norm(self.theta)

        self.Xr=np.zeros(d)

    def sample(self,A):
        N_a, _ = np.shape(A)
        UCB=np.zeros(N_a)
        norms=np.sqrt(np.diag(A@self.VtInv@A.T))
        for i in range(N_a):
            UCB[i]=np.dot(self.hat_theta,A[i])+self.beta*norms[i]
        return A[np.argmax(UCB)]

    def update(self, A, t, a_t, r_t):
        self.detV=self.detV+a_t.T @(self.VtInv) @a_t
        self.t=t+1
        if self.radi:
            self.beta=np.sqrt(self.d*np.log(1+(self.t)/(self.d*self.lamb)))+np.sqrt(self.lamb)*LA.norm(self.theta)
        else:
            self.beta=np.sqrt(np.log(self.detV/(self.delta**2)))+np.sqrt(self.lamb)*LA.norm(self.theta)
        self.Vt=self.Vt+np.outer(a_t, a_t)
        self.VtInv=self.VtInv - (np.outer(self.VtInv@a_t, self.VtInv@a_t))/(1+a_t@self.VtInv@a_t)
        self.Xr=self.Xr+a_t *r_t
        self.hat_theta = self.VtInv@self.Xr
        best_reward=np.max(A@self.theta)
        instant_reg=best_reward-np.dot(self.theta, a_t)
        if t==0:
            self.cum_regret[t]=instant_reg
        else:
            self.cum_regret[t]=self.cum_regret[t-1]+instant_reg


## Experiment class

A class designed to effectively manage the experimental environment. It takes the following major inputs:
- $s$: Sparsity level. Each experiment class has a fixed sparsity level, so one needs to create another experiment class instance to run another experiment with different sparsity level.
- $d$: Dimensionality of this linear bandit problem.
- $T$: Total number of iterations
- $\delta$: Error probability. 
- $\sigma$: The width of the uniform distribution of the noise. Noise follows $Unif(-\sigma, \sigma)$ distribution.
- $N_a$: number of arms. 
- *alg_list*: list of algorithms for this experiment. 
- *repeat*: number of repetitions. 

Methods:
- single\_exp(alg\_name, repeat\_ind): run single experiment for *alg\_name*. 
- multi\_exp(): run multiple experiment, but for each repetition all algorithms share same $\theta_*$ and $\mathcal{A}$. 
- plot\_result(): a method to plot result and save it as 'Fixed\_action\_set\_s(sparsity level).'
- generate\_theta(sparsity): generate $\theta_*$ which satisfies $\|\theta_*\|_0 = $(input sparsity)
- generate\_action\_set(): generate an action set. 

In [9]:
class Experiment():
    def __init__(self, s, d, T, delta, sigma, N_a, alg_list, theta_scale=1,  repeat=1, action_set='F'):
        self.N_alg=len(alg_list)
        self.action_set=action_set
        self.d=d
        self.s=s
        self.T=T
        self.sigma=sigma
        self.N_a = N_a
        self.delta=delta
        self.repeat=repeat
        self.alg_list=alg_list
        self.theta_scale=theta_scale
        self.cum_hist=np.zeros((len(alg_list),repeat, T))
        self.time_hist=np.zeros((len(alg_list),repeat))
        self.theta=self.generate_theta(self.s)
        self.action_seq=self.generate_action_set()

    
    def single_exp(self, alg_name, repeat_ind):
        print('Alg Name: '+alg_name+', Repeat ind:'+str(repeat_ind))
        theta=self.theta
        alg=self.create_alg_inst(theta, alg_name)
        for t in range(self.T):
            A=self.action_seq
            a_t = alg.sample(A)
            r_t = np.dot(a_t, theta)+np.random.uniform(-self.sigma,self.sigma)
            alg.update(A,t,a_t,r_t)
        self.cum_hist[self.alg_list.index(alg_name)][repeat_ind]=alg.cum_regret

    
    def multi_exp(self):
        for rep in range(self.repeat):
            self.theta=self.generate_theta(self.s)
            self.action_seq=self.generate_action_set()
            for indi, alg_name in enumerate(self.alg_list):
                tic_oneexp=time.time()
                self.single_exp(alg_name,rep)
                toc_oneexp=time.time()
                self.time_hist[indi][rep]=toc_oneexp-tic_oneexp

    def plot_result(self):
        s=0
        fonty=14
        timesteps=np.arange(self.T)
        for alg_name in self.alg_list:
            mean=np.mean(self.cum_hist[s], axis=0)
            std=np.std(self.cum_hist[s], axis=0)
            plt.plot(timesteps, mean, label=alg_name) 
            plt.fill_between(timesteps, mean-std, mean+std, alpha=0.1)
            s=s+1
        title='s='+str(self.s)
        filename='Fixed_action_set_s'+str(self.s)
        plt.title(title, weight='bold', fontsize=fonty)
        plt.xlabel('Iterations', weight='bold', fontsize=fonty)
        plt.ylabel('Cumulative Regret', weight='bold', fontsize=fonty)
        plt.legend(frameon=False, fontsize=fonty)
        plt.savefig(filename, bbox_inches='tight')
        plt.close()
    
    
    def generate_theta(self,sparsity):
        theta=np.zeros(self.d)
        signal=np.random.standard_normal(sparsity)
        #signal[:sparsity//2]=signal[:sparsity//2]*4
        signal=signal/LA.norm(signal)
        theta[:sparsity]=signal
        return theta*self.theta_scale

    
    def create_alg_inst(self, theta, alg_name):
        if alg_name=='SL_Unif':
            return SparseLinUCB(theta, self.d,self.T,self.delta, coin_dist=-2)
        elif alg_name=='SL_Known':
            return SparseLinUCB(theta, self.d,self.T,self.delta, coin_dist=-3)
        elif alg_name=='SL_Theory':
            return SparseLinUCB(theta, self.d,self.T,self.delta, coin_dist=-1)
        elif alg_name=='AL_Unif':
            return AdaLinUCB(theta, self.d,self.T,self.delta, coin_dist=0)
        elif alg_name=='AL_Theory':
            return AdaLinUCB(theta, self.d,self.T,self.delta, coin_dist=0, prior='T')
        elif alg_name=='OFUL':
            return OFUL(theta,self.d,self.T,self.delta,(self.sigma))
    
    def generate_action_set(self):
        A=np.random.standard_normal(size=(self.N_a, self.d))
        for i in range(self.N_a):
            A[i]=A[i]/LA.norm(A[i])
        return A

## Main code execution

After running the cell below, the algorithm will save the results as 'Fixed\_action\_set\_s(sparsity level).png' files. 

In [10]:
sparsity_snr=1
exp_snr=[0]*5
for j in range(5):
    exp_snr[j]=Experiment(sparsity_snr,16,10000,0.01, 1, 30, ['AL_Unif', 'AL_Theory', 'OFUL', 'SL_Unif', 'SL_Known', 'SL_Theory'], repeat=10, action_set='F')
    exp_snr[j].multi_exp()
    print('----------------------------------Sparsity: '+str(sparsity_snr)+' Done--------------------------------------')
    sparsity_snr=sparsity_snr*2
    exp_snr[j].plot_result()

Alg Name: AL_Unif, Repeat ind:0
Alg Name: AL_Theory, Repeat ind:0
Alg Name: OFUL, Repeat ind:0
Alg Name: SL_Unif, Repeat ind:0
Alg Name: SL_Known, Repeat ind:0
Alg Name: SL_Theory, Repeat ind:0
Alg Name: AL_Unif, Repeat ind:1
Alg Name: AL_Theory, Repeat ind:1
Alg Name: OFUL, Repeat ind:1
Alg Name: SL_Unif, Repeat ind:1
Alg Name: SL_Known, Repeat ind:1
Alg Name: SL_Theory, Repeat ind:1
Alg Name: AL_Unif, Repeat ind:2
Alg Name: AL_Theory, Repeat ind:2
Alg Name: OFUL, Repeat ind:2
Alg Name: SL_Unif, Repeat ind:2
Alg Name: SL_Known, Repeat ind:2
Alg Name: SL_Theory, Repeat ind:2
Alg Name: AL_Unif, Repeat ind:3
Alg Name: AL_Theory, Repeat ind:3
Alg Name: OFUL, Repeat ind:3
Alg Name: SL_Unif, Repeat ind:3
Alg Name: SL_Known, Repeat ind:3
Alg Name: SL_Theory, Repeat ind:3
Alg Name: AL_Unif, Repeat ind:4
Alg Name: AL_Theory, Repeat ind:4
Alg Name: OFUL, Repeat ind:4
Alg Name: SL_Unif, Repeat ind:4
Alg Name: SL_Known, Repeat ind:4
Alg Name: SL_Theory, Repeat ind:4
Alg Name: AL_Unif, Repeat ind:

In [13]:
np.mean(exp_snr[0].time_hist, axis=1)

array([1.98357491, 1.91569953, 0.74206486, 1.4206434 , 1.36948247,
       1.29800708])

In [14]:
np.mean(exp_snr[1].time_hist, axis=1)

array([1.89256496, 1.83572068, 0.74829917, 1.45543008, 1.43793714,
       1.40241094])

In [15]:
np.mean(exp_snr[2].time_hist, axis=1)

array([2.19421396, 2.16109395, 0.85595367, 1.67763534, 1.69114707,
       1.66488755])

In [16]:
np.mean(exp_snr[3].time_hist, axis=1)

array([1.81707954, 1.78819101, 0.69890943, 1.31845014, 1.37100842,
       1.39613771])

In [17]:
np.mean(exp_snr[4].time_hist, axis=1)

array([1.68315079, 1.70087025, 0.68356259, 1.25484254, 1.29798574,
       1.29368417])